<a href="https://colab.research.google.com/github/RSDUNN0809/create_sqlite_database_in_python/blob/main/Create_a_SQLite3_Database_in_Google_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to Create a SQLite3 Database in Python in Just 4 Short Steps Using Google Colab

IG: @dunnford_does_data

YouTube: @RyanSDunn


## Connect to your Google Drive

In [1]:
#connect to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Import required libraries:

In [2]:
import pandas as pd
import sqlite3
import csv

# Set option to see all columns in output (I like to do this - you don't have to)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',None)

## 1. Create a new connection to a filepath and create the new_test.db sqlite database

In [3]:
# Connect to a sqlite3 database, or create one if it does not exist
conn = sqlite3.connect('/content/drive/My Drive/Ryan Other Data Science Projects/Create a SQLite3 Database in Python/new_test.db')

# Create the cursor object
cursor = conn.cursor()

# Save the changes to the database
conn.commit()

# Close the cursor and connection objects
cursor.close()
conn.close()

## 2. Import a .csv file to upload into the SQLite database

In [4]:
# Import the the .csv file from a google drive folder path
raw_df = pd.read_csv('/content/drive/MyDrive/Ryan Other Data Science Projects/Fraud Detection with Translatable Features/card_transdata.csv', 
                     header=0)
df = raw_df

# Quickly view shape and head of dataframe
print(df.shape)
df.head()

(1000000, 8)


,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,used_chip,used_pin_number,online_order,fraud
0,57.877857,0.311140,1.945940,1.0,1.0,0.0,0.0,0.0
1,10.829943,0.175592,1.294219,1.0,0.0,0.0,0.0,0.0
2,5.091079,0.805153,0.427715,1.0,0.0,0.0,1.0,0.0
3,2.247564,5.600044,0.362663,1.0,1.0,0.0,1.0,0.0
4,44.190936,0.566486,2.222767,1.0,1.0,0.0,1.0,0.0


## 3. Write the contents of the dataframe to your new_test.db
(In this example, we're going to name the table "transaction_table")

In [5]:
# Connect to a sqlite3 database, or create one if it does not exist
conn = sqlite3.connect('/content/drive/My Drive/Ryan Other Data Science Projects/Create a SQLite3 Database in Python/new_test.db')

# Create the cursor object
cursor = conn.cursor()

# Write the contents of the DataFrame to a SQLite table, with column headers
df.to_sql('transaction_table', conn, if_exists='replace', index=True)

# Save changes and commit to database
conn.commit()

# Close the cursor and connection objects
cursor.close()
conn.close()

## 4. Now, it's time to query our SQLite database and save the output to a dataframe

In [6]:
# connect to the database
conn = sqlite3.connect('/content/drive/My Drive/Ryan Other Data Science Projects/Create a SQLite3 Database in Python/new_test.db')

##### Query transaction_table data ##### 
# Create cursor object and query table
cursor = conn.cursor()
cursor.execute("""SELECT TT.distance_from_home, TT.repeat_retailer, TT.fraud 
                  FROM transaction_table as TT 
                  WHERE TT.distance_from_home >= 50
                  ORDER BY TT.distance_from_home DESC
                  LIMIT 10""")

# Identify the column names for dataframe
col_names = [desc[0] for desc in cursor.description]

# Fetch all rows from the query and store in dataframe
rows = cursor.fetchall()

# Create the transactions dataframe from the select rows and col_names from sqlite
transactions_df = pd.DataFrame(rows, columns = col_names)

# Close the cursor and connection objects
cursor.close()
conn.close()

In [7]:
# View the contents of the created dataframe from the SQL query
print(transactions_df.shape)
transactions_df.head()

(10, 3)


,distance_from_home,repeat_retailer,fraud
0,10632.723672,1.0,1.0
1,8777.136420,1.0,0.0
2,5797.972589,1.0,0.0
3,4823.638452,1.0,0.0
4,4601.011222,1.0,0.0


## Conversly, you may have multiple dataframes from multiple .csv's
And the process is just as easy!

In [8]:
# Import the .csv documents and save to dataframes
transformed_df = pd.read_csv('/content/drive/MyDrive/MS-ADS Capstone Project/data/data for models/Copy of transformed_df.csv',header=0)
non_transformed_df = pd.read_csv('/content/drive/MyDrive/MS-ADS Capstone Project/data/data for models/Copy of non_transformed_df.csv',header=0)

# Check dataframe dimensions
print(transformed_df.shape)
print(non_transformed_df.shape)

(4410, 57)
(4410, 59)


In [9]:
# Connect to a sqlite3 database, or create one if it does not exist
conn = sqlite3.connect('/content/drive/My Drive/MS-ADS Capstone Project/data/hr_case_study.db')

# Create the cursor object
cursor = conn.cursor()

# Write the contents of the DataFrame to a SQLite table, with column headers
transformed_df.to_sql('transformed_data', conn, if_exists='replace', index=True)
non_transformed_df.to_sql('non_transformed_data', conn, if_exists='replace', index=True)

# Save changes and commit to database
conn.commit()

# Close the cursor and connection objects
cursor.close()
conn.close()

In [10]:
# Connect to the database
conn = sqlite3.connect('/content/drive/My Drive/MS-ADS Capstone Project/data/hr_case_study.db')

##### Query general_data table ##### 
# Create cursor object and query table
cursor = conn.cursor()
cursor.execute("""SELECT * FROM transformed_data as td
                  LEFT JOIN non_transformed_data as nt
                    ON td.EmployeeID = nt.EmployeeID
               """)

# Identify the column names for dataframe
col_names = [desc[0] for desc in cursor.description]

# Fetch all rows from the query and store in dataframe
rows = cursor.fetchall()
sql_data= pd.DataFrame(rows, columns = col_names)

# Close the cursor and connection objects
cursor.close()
conn.close()

In [11]:
sql_data.head()

,index,Unnamed: 0,Education,EmployeeID,JobLevel,NumCompaniesWorked,JobInvolvement,PerformanceRating,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,time_id,mean_time,median_time,max_time,num_times_over_9hours,num_times_over_11hours,%_under_8,%_over_9,%_over_11,clipped_TrainingTimesLastYear,clipped_NumCompaniesWorked,log_Age,log_StockOptionLevel,log_YearsSinceLastPromotion,log_TotalWorkingYears,log_YearsAtCompany,log_MonthlyIncome,log_PercentSalaryHike,log_DistanceFromHome,log_clipped_YearsWithCurrentManager,log_clipped_num_times_less_8hours,log_total_days,log_num_times_over_9hours,log_num_times_over_11hours,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,Department_Research & Development,Department_Sales,EducationField_Life Sciences,EducationField_Marketing,EducationField_Medical,EducationField_Other,EducationField_Technical Degree,JobRole_Human Resources,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Married,MaritalStatus_Single,Attrition_Yes,Gender_Male,log_AgeBeganWorking,log_WorkingDaysPerYear,index,Unnamed: 0,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeID,Gender,JobLevel,JobRole,MaritalStatus,MonthlyIncome,NumCompaniesWorked,Over18,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrentManager,JobInvolvement,PerformanceRating,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,time_id,mean_time,median_time,max_time,num_times_less_8hours,num_times_over_9hours,num_times_over_11hours,total_days,%_under_8,%_over_9,%_over_11,clipped_StockOptionLevel,clipped_YearsWithCurrentManager,clipped_TrainingTimesLastYear,clipped_NumCompaniesWorked,clipped_num_times_less_8hours,log_Age,log_StockOptionLevel,log_YearsSinceLastPromotion,log_TotalWorkingYears,log_YearsAtCompany,log_MonthlyIncome,log_PercentSalaryHike,log_DistanceFromHome,log_clipped_YearsWithCurrentManager,log_clipped_num_times_less_8hours,log_total_days,log_num_times_over_9hours,log_num_times_over_11hours
0,0,0,0.25,1,1,1.0,0.666667,0.0,0.666667,1.000000,0.333333,1,0 days 07:22:26.896551724,0 days 07:23:00,0 days 07:57:00,0,0,1.0,0.0,0.0,1.00,0.125,0.863147,0.000000,0.00,0.186652,0.186652,0.858755,0.000000,0.462607,0.000000,0.992462,0.847162,0.0,0.0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.858668,0.926953,0,0,51.0,No,Travel_Rarely,Sales,6,2,Life Sciences,1,1,Female,1,Healthcare Representative,Married,131160,1.0,Y,11,8,0,1.0,6,1,0,0,3,3,3,4,2,1,0 days 07:22:26.896551724,0 days 07:23:00,0 days 07:57:00,235,0,0,235,1.0,0.0,0.0,0,0.0,5.0,1.0,235.0,1.716003,0.000000,0.00000,0.301030,0.301030,5.117805,0.000000,0.462607,0.000000,0.992462,0.847162,0.0,0.0
1,1,1,0.00,2,1,0.0,0.333333,1.0,0.666667,0.333333,1.000000,2,0 days 07:43:03.813559322,0 days 07:43:00,0 days 08:34:00,0,0,1.0,0.0,0.0,0.50,0.000,0.446915,0.301030,0.25,0.482489,0.482489,0.476600,0.896477,0.629511,0.671188,0.959652,0.791635,0.0,0.0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0.305588,0.626674,1,1,31.0,Yes,Travel_Frequently,Research & Development,10,1,Life Sciences,1,2,Female,1,Research Scientist,Single,41890,0.0,Y,23,8,1,5.0,3,5,1,4,2,4,3,2,4,2,0 days 07:43:03.813559322,0 days 07:43:00,0 days 08:34:00,196,0,0,196,1.0,0.0,0.0,1,4.0,3.0,0.0,196.0,1.505150,0.301030,0.30103,0.778151,0.778151,4.622121,0.896477,0.629511,0.671188,0.959652,0.791635,0.0,0.0
2,2,2,0.75,3,4,1.0,0.666667,0.0,0.333333,0.333333,0.000000,3,0 days 07:00:48.595041322,0 days 06:59:00,0 days 07:59:00,0,0,1.0,0.0,0.0,0.25,0.125,0.473296,0.602060,0.00,0.482489,0.482489,0.988573,0.372072,0.811368,0.578130,1.000000,0.859920,0.0,0.0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0.337215,0.663684,2,2,32.0,No,Travel_Frequently,Research & Development,17,4,Other,1,3,Male,4,Sales Executive,Married,193280,1.0,Y,15,8,3,5.0,2,5,0,3,3,3,2,2,1,3,0 d